In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import  Dense, Activation,Dropout, Flatten
from sklearn.linear_model import Ridge, Lasso, SGDRegressor
from sklearn.svm import NuSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score,mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
train=pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
test=pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')

In [ ]:
sample=pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv')
sample.head(5)

In [ ]:
train.head(3)

In [ ]:
train=train.iloc[0:, 1:]

In [ ]:
train.head()

In [ ]:
test=test.iloc[0:, 1:]

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.isna().sum()

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
var_thres=VarianceThreshold(threshold=0)

In [ ]:
var_thres.fit(train)

In [ ]:
constant_colums=[i for i in train.columns if i not in train.columns[var_thres.get_support()]]

In [ ]:
constant_colums

In [ ]:
Xtrain=train[['breath_id','R','C','time_step','u_in','u_out']]

In [ ]:
Ytrain=train[['pressure']]

# dropping feature using correlation

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(12,10))
corr=Xtrain.corr()
sns.heatmap(corr, annot=True)
plt.show()

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:

corr_features=correlation(Xtrain, threshold=0.7)

In [ ]:
Xtrain.drop(corr_features,axis=1)
test.drop(corr_features,axis=1)

In [ ]:
for i in train.columns:
    print(train[i].unique)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc=StandardScaler()

In [ ]:
train1 =pd.DataFrame(sc.fit_transform(Xtrain))

In [ ]:
test1=pd.DataFrame(sc.fit_transform(test))

In [ ]:
train1

# building model for training and testing of the dataset

In [ ]:
NN_model = Sequential()
NN_model.add(Dense(256, kernel_initializer='normal',input_dim = train1.shape[1], activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(512, kernel_initializer='normal',activation='linear'))
NN_model.add(Dense(1, kernel_initializer='normal'))
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
#checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
#checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
#callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(train1, Ytrain, epochs=5, batch_size=32, validation_split = 0.2)
             #, callbacks=callbacks_list)


In [ ]:
#wights_file = 'Weights-478--18738.19831.hdf5' # choose the best checkpoint 
#NN_model.load_weights(wights_file) # load it
#NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [ ]:
predictions = NN_model.predict(test1)

In [ ]:
predictions

In [ ]:
train1.shape

In [ ]:
train_x=train1.values.reshape(train1.shape[0],1,train1.shape[1])

In [ ]:
"""model = Sequential()
model.add(LSTM(units=400, activation='relu',return_sequences=True,input_shape=(train_x.shape[1],train_x.shape[2])))
model.add(Dense(150))
model.add(LSTM(units=200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])"""

In [ ]:
#history=model.fit(train_x,Ytrain,epochs=2,batch_size=128, validation_split=0.3)

In [ ]:
"""plt.figure(figsize=(20,6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show()"""

In [ ]:
#test_x=test1.values.reshape(test1.shape[0],1,test1.shape[1])

In [ ]:
#y_output=model.predict(test_x)

In [ ]:
#y_output

In [ ]:
test=pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')

In [ ]:
sub=pd.DataFrame()
sub['id']=test['id']
sub['pressure']=pd.Series(predictions.flatten())

In [ ]:
sub.to_csv('submission.csv', index=False)


In [ ]:
#from sklearn.ensemble import RandomForestRegressor

In [ ]:
#regr = RandomForestRegressor(max_depth=2, random_state=0)
#regr.fit(train1, Ytrain)

In [ ]:
#ot=regr.predict(test1)

In [ ]:
#sub=pd.DataFrame()
#sub['id']=test['id']
#sub['pressure']=pd.Series(ot)

In [ ]:
#sub.to_csv('sub1.csv', index=False)

In [ ]:
#ot

In [ ]:
train['pressure'].plot()

In [ ]:
sub['pressure'].plot()